## Въведение 

Този урок ще обхване: 
- Какво е извикване на функция и неговите случаи на използване 
- Как да създадете извикване на функция с помощта на OpenAI 
- Как да интегрирате извикване на функция в приложение 

## Цели на обучението 

След завършване на този урок ще знаете как да и ще разберете: 

- Целта на използването на извикване на функция 
- Настройване на извикване на функция с помощта на OpenAI Service 
- Проектиране на ефективни извиквания на функции за случая на използване на вашето приложение


## Разбиране на извикванията на функции

За този урок искаме да създадем функция за нашия образователен стартъп, която позволява на потребителите да използват чатбот за намиране на технически курсове. Ще препоръчваме курсове, които отговарят на тяхното ниво на умения, текущата роля и интересуващата ги технология.

За да завършим това, ще използваме комбинация от:
 - `OpenAI` за създаване на чат изживяване за потребителя
 - `Microsoft Learn Catalog API`, за да помогнем на потребителите да намерят курсове въз основа на заявката на потребителя
 - `Function Calling`, за да вземем заявката на потребителя и да я изпратим към функция за извършване на API заявката.

За да започнем, нека разгледаме защо бихме искали да използваме извикване на функция на първо място:

print("Съобщения в следващата заявка:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # получаване на нов отговор от GPT, където може да види отговора на функцията


print(second_response.choices[0].message)


### Защо извикване на функции

Ако сте завършили някой друг урок в този курс, вероятно разбирате силата на използването на Големи езикови модели (LLMs). Надяваме се, че също така можете да видите някои от техните ограничения.

Извикването на функции е функция на OpenAI Service, предназначена да адресира следните предизвикателства:

Несъгласувано форматиране на отговорите:
- Преди извикването на функции, отговорите от голям езиков модел бяха неструктурирани и несъгласувани. Разработчиците трябваше да пишат сложен код за валидиране, за да обработват всяка вариация в изхода.

Ограничена интеграция с външни данни:
- Преди тази функция беше трудно да се включат данни от други части на приложението в контекста на чат.

Чрез стандартизиране на форматирането на отговорите и осигуряване на безпроблемна интеграция с външни данни, извикването на функции опростява разработката и намалява нуждата от допълнителна логика за валидиране.

Потребителите не можеха да получат отговори като "Какво е текущото време в Стокхолм?". Това е така, защото моделите бяха ограничени до времето, на което са обучени данните.

Нека разгледаме примера по-долу, който илюстрира този проблем:

Да кажем, че искаме да създадем база данни със студентски данни, за да можем да им предложим подходящия курс. По-долу имаме две описания на студенти, които са много сходни по данните, които съдържат.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Искаме да изпратим това на LLM, за да анализира данните. Това по-късно може да се използва в нашето приложение за изпращане към API или за съхранение в база данни.

Нека създадем два идентични подсказки, в които да инструктираме LLM каква информация ни интересува:


Искаме да изпратим това на LLM, за да анализира частите, които са важни за нашия продукт. Така можем да създадем два идентични подсказки, за да инструктираме LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


След създаването на тези два подсказки, ще ги изпратим към LLM, като използваме `openai.ChatCompletion`. Съхраняваме подсказката в променливата `messages` и задаваме ролята на `user`. Това е, за да се имитира съобщение от потребител, написано към чатбот.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Сега можем да изпратим и двете заявки към LLM и да разгледаме получения отговор.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Въпреки че подканите са еднакви и описанията са подобни, можем да получим различни формати на свойството `Grades`.

Ако изпълните горната клетка няколко пъти, форматът може да бъде `3.7` или `3.7 GPA`.

Това е така, защото LLM приема неструктурирани данни под формата на написаната подканваща команда и връща също неструктурирани данни. Трябва да имаме структуриран формат, за да знаем какво да очакваме при съхранение или използване на тези данни.

Чрез използване на функционално извикване можем да сме сигурни, че ще получим обратно структуриран формат на данните. При използване на функционално извикване LLM всъщност не извиква или изпълнява никакви функции. Вместо това създаваме структура, която LLM трябва да следва за своите отговори. След това използваме тези структурирани отговори, за да знаем коя функция да изпълним в нашите приложения.


![Диаграма на потока на извикване на функция](../../../../translated_images/Function-Flow.083875364af4f4bb.bg.png)


След това можем да вземем това, което функцията връща, и да го изпратим обратно на LLM. LLM след това ще отговори, използвайки естествен език, за да отговори на запитването на потребителя.


### Примери за използване на извиквания на функции

**Извикване на външни инструменти**  
Чатботовете са отлични в предоставянето на отговори на въпроси от потребителите. Чрез използване на извикване на функции, чатботовете могат да използват съобщения от потребителите, за да изпълнят определени задачи. Например, студент може да помоли чатбота да „Изпрати имейл на моя преподавател, че имам нужда от повече помощ по този предмет“. Това може да направи извикване на функцията `send_email(to: string, body: string)`

**Създаване на API или заявки към база данни**  
Потребителите могат да намерят информация, използвайки естествен език, който се преобразува във форматирана заявка или API повикване. Пример за това може да бъде учител, който пита „Кои са студентите, които са завършили последното задание“, което може да извика функция с име `get_completed(student_name: string, assignment: int, current_status: string)`

**Създаване на структурирани данни**  
Потребителите могат да вземат блок текст или CSV и да използват LLM, за да извлекат важна информация от него. Например, студент може да преобразува статия от Уикипедия за мирни споразумения, за да създаде AI флашкарти. Това може да се направи чрез използване на функция, наречена `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Създаване на първото ви извикване на функция

Процесът на създаване на извикване на функция включва 3 основни стъпки:  
1. Извикване на Chat Completions API с списък на вашите функции и съобщение от потребителя  
2. Прочитане на отговора на модела, за да се извърши действие, т.е. изпълнение на функция или API извикване  
3. Направете друго извикване към Chat Completions API с отговора от вашата функция, за да използвате тази информация за създаване на отговор към потребителя.


![Поток на извикване на функция](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.bg.png)


### Елементи на извикване на функция 

#### Вход от потребителя 

Първата стъпка е да се създаде съобщение от потребителя. Това може да бъде динамично зададено чрез вземане на стойността от текстово поле или можете да зададете стойност тук. Ако това е първият път, когато работите с Chat Completions API, трябва да дефинираме `role` и `content` на съобщението. 

`role` може да бъде или `system` (създаване на правила), `assistant` (моделът) или `user` (крайният потребител). За извикване на функция ще зададем това като `user` и примерен въпрос. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Създаване на функции.

След това ще дефинираме функция и параметрите на тази функция. Тук ще използваме само една функция, наречена `search_courses`, но можете да създадете и няколко функции.

**Важно** : Функциите са включени в системното съобщение към LLM и ще бъдат включени в броя на наличните токени, с които разполагате.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Дефиниции** 

Структурата на дефиницията на функцията има няколко нива, всяко със свои собствени свойства. Ето разбивка на вложената структура:

**Основни свойства на функцията:**

`name` - Името на функцията, която искаме да бъде извикана. 

`description` - Това е описанието на начина, по който функцията работи. Тук е важно да бъдете конкретни и ясни. 

`parameters` - Списък с стойности и формат, които искате моделът да генерира в своя отговор. 

**Свойства на обекта Parameters:**

`type` - Типът данни на обекта parameters (обикновено "object")

`properties` - Списък със специфичните стойности, които моделът ще използва за своя отговор. 

**Свойства на отделните параметри:**

`name` - Имплицитно дефинирано чрез ключа на свойството (например "role", "product", "level")

`type` - Типът данни на конкретния параметър (например "string", "number", "boolean") 

`description` - Описание на конкретния параметър. 

**Опционални свойства:**

`required` - Масив, изброяващ кои параметри са задължителни за изпълнението на извикването на функцията.


### Извикване на функцията  
След като дефинираме функция, сега трябва да я включим в извикването на Chat Completion API. Правим това, като добавим `functions` към заявката. В този случай `functions=functions`. 

Има и опция да зададем `function_call` на `auto`. Това означава, че ще позволим на LLM да реши коя функция трябва да бъде извикана въз основа на съобщението на потребителя, вместо да я задаваме ние.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Сега нека разгледаме отговора и да видим как е форматиран:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Можете да видите, че името на функцията е извикано и от съобщението на потребителя LLM успя да намери данните, които да отговарят на аргументите на функцията.


## 3.Интегриране на извиквания на функции в приложение. 


След като тествахме форматирания отговор от LLM, сега можем да го интегрираме в приложение. 

### Управление на потока 

За да го интегрираме в нашето приложение, нека предприемем следните стъпки: 

Първо, нека направим извикването към услугите на OpenAI и съхраним съобщението в променлива, наречена `response_message`. 


In [ ]:
response_message = response.choices[0].message

Сега ще дефинираме функцията, която ще извика Microsoft Learn API, за да получи списък с курсове:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Като най-добра практика, след това ще видим дали моделът иска да извика функция. След това ще създадем една от наличните функции и ще я съпоставим с функцията, която се извиква.  
След това ще вземем аргументите на функцията и ще ги свържем с аргументите от LLM.

Накрая ще добавим съобщението за извикване на функцията и стойностите, които бяха върнати от съобщението `search_courses`. Това дава на LLM цялата необходима информация, за да отговори на потребителя, използвайки естествен език.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Сега ще изпратим актуализираното съобщение до LLM, за да получим отговор на естествен език вместо отговор във формат JSON от API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code Challenge 

Страхотна работа! За да продължите обучението си по OpenAI Function Calling, можете да изградите: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Повече параметри на функцията, които могат да помогнат на учащите да намерят повече курсове. Можете да намерите наличните API параметри тук:  
 - Създайте друго извикване на функция, което приема повече информация от учащия, като например техния роден език  
 - Създайте обработка на грешки, когато извикването на функцията и/или извикването на API не връщат подходящи курсове  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от отговорност**:  
Този документ е преведен с помощта на AI преводаческа услуга [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи могат да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е недоразумения или неправилни тълкувания, произтичащи от използването на този превод.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
